In [ ]:
from time import time

import numpy as np
import scipy.stats as stats
import dask.array as da

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
import sklearn

import dask_ml
from dask_ml.datasets import make_classification
from dask_ml.wrappers import Incremental
from dask_ml.model_selection import HyperbandCV
import dask_searchcv

from time import sleep
dask_ml.__file__

In [ ]:
from distributed import Client, LocalCluster
procs, threads = 8, 11
num_workers = procs * threads
cluster = LocalCluster(n_workers=procs, threads_per_worker=threads)
client = Client(cluster)

In [ ]:
# n, d = int(10e3), int(100)
# X, y = make_classification(n_features=d, n_samples=n,
#                            chunks=(n // 10, d))
# classes = da.unique(y)

# kwargs = dict(loss='hinge', penalty='elasticnet',
#               max_iter=1.0, warm_start=True)
# model = Incremental(SGDClassifier(), **kwargs)
# params = {'alpha': np.logspace(-4, 0, num=1000),
#           'l1_ratio': stats.uniform(0, 1),
#           'average': [True, False]}
# partial_fit_kwargs = {'classes': classes}

from dask_ml.utils import ConstantFunction
model = ConstantFunction()
params = {'value': stats.uniform(0, 1)}
sleep = 0.1
partial_fit_kwargs = {'sleep': sleep}

In [ ]:
n, d = int(1000), int(200)
X, y = make_classification(n_features=d, n_samples=n,
                           chunks=(n//10, d))
alg = HyperbandCV(model, params, max_iter=81)

info = alg.info()

train_time = sleep * info['total_partial_fit_calls']
best_time = train_time / num_workers
print(f"Fastest possible time = {best_time}s")

In [ ]:
start = time()
alg.fit(X, y, **partial_fit_kwargs)
actual_time = time() - start
print(f"Actual time = {actual_time}")

In [ ]:
import pandas as pd
df = pd.DataFrame(alg.history_)
print(df.bracket.max())
print(df.bracket_iter.unique())
df.tail()
# df[df.mean_test_score > 0.95]

In [ ]:
from altair import Chart
Chart(df).mark_circle().encode(
    x='bracket_iter', row='bracket', y='mean_test_score')